In [1]:
# Importar Bibliotecas
import pandas as pd
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, col


In [ ]:
"""Crea una sesión de Spark con el nombre de la aplicación "Transformación de Datos y Unión de Geolocalización", 
habilitando el soporte para Hive, y la inicializa o reutiliza si ya existe"""
spark = SparkSession \
    .builder \
    .appName("Transformación de Datos y Unión de Geolocalización") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
# Leer varios archivos Parquet para crear DataFrames
geo = spark.read.parquet('/home/felipe/despliegue_analytica/files_parquet/olist_geolocation_dataset.parquet')
zip_code = spark.read.parquet('/home/felipe/despliegue_analytica/files_parquet/log.parquet')

In [ ]:
# Crear una nueva columna llamada 'cep_prefix' que contiene los primeros 5 dígitos del campo 'cep'
zip_code = zip_code.withColumn("cep_prefix", substring(col("cep"), 1, 5))
# Cambiar el nombre de la columna 'geolocation_zip_code_prefix' a 'cep_prefix'
geo = geo.withColumnRenamed("geolocation_zip_code_prefix", "cep_prefix")
# Realizar un join entre los DataFrames 'geo' y 'zip_code' utilizando 'cep_prefix' como clave de unión
joined_df = geo.join(zip_code, "cep_prefix", "inner")
# Seleccionar y renombrar columnas para crear un nuevo DataFrame 'result_df'
result_df = joined_df.select("cep", "geolocation_city", "geolocation_state") \
                     .withColumn("cep_5_digitos", substring(col("cep"), 1, 5))
# Mostrar el resultado final sin truncar los valores
result_df.show(truncate=False)

In [ ]:
# Modificar los nombres de las columnas en 'result_df' y crear un DataFrame final llamado 'result_df_final'
result_df_final = result_df.withColumnRenamed("logradouro", "street") \
                   .withColumnRenamed("geolocation_city", "city") \
                   .withColumnRenamed("geolocation_state", "uf") \
                   .withColumnRenamed("cep_5_digitos", "cep_prefix")

In [ ]:
# Guarda el archivo en formato Parquet en la carpeta "files_parquet"
result_df_final.write.mode('overwrite').parquet('/home/felipe/despliegue_analytica/files_parquet/geolocation_correios.parquet')